In [4]:
import pandas as pd
from glob import glob
import numpy as np
from easydict import EasyDict as edict
import os
import time
import sys
from tabulate import tabulate

sys.path.insert(0,os.path.expanduser('~/git/torchseg'))
print(sys.path)

from utils.config import load_config
from utils.summary_to_csv import config_to_log,load_log,edict_to_pandas,today_log,recent_log

def summary(rootpath,tags,filter_str=None,):
    config_files=glob(os.path.join(rootpath,'**','config.txt'),recursive=True)
    if filter_str == 'today':
        config_files=today_log(config_files)
    elif filter_str == 'recent':
        config_files=recent_log(config_files)
    elif filter_str is not None:
        config_files=[f for f in config_files if f.find(filter_str)>=0]
    
    tasks=pd.DataFrame()
    for cfg in config_files:
        log=config_to_log(cfg)
        if log is not None:
            ed=load_config(cfg)
            metrics=load_log(log,tags)
            for key,value in metrics.items():
                ed[key]=value
            
            ed['dir']=cfg
            log_time=''
            for s in cfg.split(os.sep):
                if s.find('___')>=0:
                    log_time=s
                    break
            ed['log_time']=log_time
            
            task=edict_to_pandas(ed)
            tasks=tasks.append(task,ignore_index=True,sort=False)
        else:
            print('cannot find log file for',cfg)
            
    return tasks

def dump(tags=['train/fmeasure','val/fmeasure'],
        rootpath=os.path.expanduser('~/tmp/logs/motion'),
        notes=['motion_fcn','motion_sparse','motion_psp'],
        note_gtags=[['upsample_layer','freeze_layer','ignore_outOfRoi'],
                    ['upsample_layer','freeze_layer','sparse_ratio'],
                   ['upsample_layer','freeze_layer','ignore_outOfRoi']],
        descripe=['epoch'],
        delete_nan=False,
        dump_group=False,
        dump_dir=True):
    dir_tags=[tags[1],'dir']
    for note,ntag in zip(notes,note_gtags):
        show_tags=[]
        show_tags=tags.copy()
        group_tags=['note']+ntag
        show_tags+=group_tags
        show_tags+=descripe
        tasks=summary(rootpath,tags,note)
        if tasks.empty:
            print(note,ntag,'task is empty')
            continue

        # remove empty dirs
        if delete_nan:
            dirs=tasks[tasks[tags[1]].isna()]['dir'].tolist()
            tasks=tasks[tasks[tags[1]].notna()]
            for dir in dirs:
                print(os.path.dirname(dir))
                os.system('rm -rf {}'.format(os.path.dirname(dir)))

        #print(tasks[show_tags].groupby(group_tags).max().to_string())
        if dump_group:
            print(tabulate(tasks[show_tags].groupby(group_tags).mean().sort_values(tags[1]),tablefmt='pipe',headers='keys'))
            print('\n')
            print(tabulate(tasks[[tags[1]]+group_tags].groupby(group_tags).agg([np.mean,np.std,np.max]),tablefmt='pipe',headers='keys'))
            print('\n')
        
        if note=='recent':
            print(tabulate(tasks[show_tags].sort_values('log_time'),tablefmt='pipe',headers='keys'))
        else:
            print(tabulate(tasks[show_tags].sort_values(tags[1]),tablefmt='pipe',headers='keys'))
        print('\n')
        if dump_dir:
            print(tabulate(tasks[dir_tags].sort_values(tags[1]),tablefmt='pipe',headers='keys'))
            print('\n')

['/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '', '/home/yzbx/git/torchseg/notebooks', '/home/yzbx/git/gnu/models/research', '/home/yzbx/git/gnu/models/research/slim', '/home/yzbx/bin/miniconda3/envs/new/lib/python36.zip', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/lib-dynload', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/cityscapesscripts-1.0.0-py3.6-linux-x86_64.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg', '/home/yzbx/bin/miniconda3/envs/new/lib/python3.6/site-packages/IPython/extensions', '/home/yzbx/.ipython']


In [5]:
tags=['train/fmeasure','val/fmeasure']
rootpath=os.path.expanduser('~/tmp/logs/motion')
notes=['task011','task012','task013']
note_gtags=[['dataset','net_name'],
           ['frame_gap','net_name'],
           ['pose_mask_reg']]
dump(tags=tags,rootpath=rootpath,notes=notes,note_gtags=note_gtags,descripe=['epoch'],delete_nan=False,dump_group=True)

|                                                          |   train/fmeasure |   val/fmeasure |   epoch |
|:---------------------------------------------------------|-----------------:|---------------:|--------:|
| ('task011_res_u3_d5_f0', 'cdnet2014', 'motion_unet')     |         0.948644 |       0.668256 |      30 |
| ('task011_res_u2_d4_f0', 'cdnet2014', 'motion_unet_stn') |         0.954001 |       0.677904 |      30 |
| ('task011_res_u2_d5_f0', 'cdnet2014', 'motion_unet_stn') |         0.935422 |       0.69845  |      30 |
| ('task011_res_u3_d5_f0', 'cdnet2014', 'motion_unet_stn') |         0.902884 |       0.700196 |      30 |
| ('task011_res_u3_d4_f0', 'FBMS', 'motion_unet_stn')      |         0.969338 |       0.712433 |      30 |
| ('task011_res_u3_d4_f0', 'cdnet2014', 'motion_unet')     |         0.955174 |       0.71314  |      30 |
| ('task011_res_u2_d5_f0', 'cdnet2014', 'motion_unet')     |         0.95323  |       0.715807 |      30 |
| ('task011_res_u2_d4_f0', 'cdnet2014

|                                            |   train/fmeasure |   val/fmeasure |   epoch |
|:-------------------------------------------|-----------------:|---------------:|--------:|
| ('task012_bs8_f5', 5, 'motion_unet_stn')   |         0.964169 |       0.783302 |      30 |
| ('task012_bs8_f10', 10, 'motion_unet_stn') |         0.961857 |       0.797964 |      30 |
| ('task012_bs2_f20', 20, 'motion_unet')     |         0.970751 |       0.826775 |      30 |
| ('task012_bs4_f10', 10, 'motion_unet')     |         0.964551 |       0.82944  |      30 |
| ('task012_bs8_f20', 20, 'motion_unet_stn') |         0.966438 |       0.831978 |      30 |
| ('task012_bs2_f10', 10, 'motion_unet')     |         0.956664 |       0.832967 |      30 |
| ('task012_bs8_f5', 5, 'motion_unet')       |         0.964449 |       0.833654 |      30 |
| ('task012_bs8_f20', 20, 'motion_unet')     |         0.962565 |       0.836517 |      30 |
| ('task012_bs4_f5', 5, 'motion_unet_stn')   |         0.959662 |     

|                       |   train/fmeasure |   val/fmeasure |   epoch |
|:----------------------|-----------------:|---------------:|--------:|
| ('task013_pmr', -2.0) |         0.971876 |       0.805376 |      30 |
| ('task013_pmr', 0.0)  |         0.970377 |       0.81245  |      30 |
| ('task013_pmr', -1.0) |         0.971957 |       0.814936 |      30 |
| ('task013_pmr', 1.0)  |         0.97095  |       0.817578 |      30 |


|                       |   ('val/fmeasure', 'mean') |   ('val/fmeasure', 'std') |   ('val/fmeasure', 'amax') |
|:----------------------|---------------------------:|--------------------------:|---------------------------:|
| ('task013_pmr', -2.0) |                   0.805376 |               0.00190457  |                   0.806722 |
| ('task013_pmr', -1.0) |                   0.814936 |               0.01727     |                   0.827147 |
| ('task013_pmr', 0.0)  |                   0.81245  |               0.000752195 |                   0.812982 |
| ('ta

In [7]:
tags=['train/fmeasure','val/fmeasure']
rootpath=os.path.expanduser('~/tmp/logs/motion')
notes=['today','recent']
note_gtags=[['log_time','net_name'],
           ['log_time','net_name']]
dump(tags=tags,rootpath=rootpath,notes=notes,note_gtags=note_gtags,descripe=['epoch'],delete_nan=False,dump_group=False)

today ['log_time', 'net_name'] task is empty
/home/yzbx/tmp/logs/motion/motion_unet/segtrackv2/segtrack/2019-03-12___16-58-17
|    |   train/fmeasure |   val/fmeasure | note                  | log_time              | net_name        |   epoch |
|---:|-----------------:|---------------:|:----------------------|:----------------------|:----------------|--------:|
|  0 |         0.928518 |       0.686802 | task004_sr0.9         | 2019-03-07___00-59-02 | motion_sparse   |      30 |
|  1 |         0.965266 |       0.811781 | task005_u0_d5_f0      | 2019-03-07___11-35-28 | motion_unet     |      30 |
|  2 |         0.955551 |       0.786291 | task005_u1_d5_f0      | 2019-03-07___11-54-45 | motion_unet     |      30 |
|  3 |         0.968709 |       0.813394 | task005_u2_d5_f0      | 2019-03-07___12-12-26 | motion_unet     |      30 |
|  4 |         0.957786 |       0.700421 | test_stn              | 2019-03-07___12-44-16 | motion_unet_stn |      30 |
|  5 |         0.963043 |       0.773411 

|    |   val/fmeasure | dir                                                                                                         |
|---:|---------------:|:------------------------------------------------------------------------------------------------------------|
| 99 |       0.448145 | /home/yzbx/tmp/logs/motion/motion_unet/segtrackv2/segtrack/2019-03-12___16-58-56/config.txt                 |
| 16 |       0.647085 | /home/yzbx/tmp/logs/motion/motion_sparse/cdnet2014/task006_sr0.9/2019-03-07___15-12-54/config.txt           |
| 39 |       0.661018 | /home/yzbx/tmp/logs/motion/motion_unet/cdnet2014/task007_u4_d5_f0/2019-03-08___17-52-47/config.txt          |
| 59 |       0.668256 | /home/yzbx/tmp/logs/motion/motion_unet/cdnet2014/task011_res_u3_d5_f0/2019-03-10___19-48-47/config.txt      |
| 45 |       0.669251 | /home/yzbx/tmp/logs/motion/motion_unet_stn/cdnet2014/task007_u4_d5_f0/2019-03-08___18-46-58/config.txt      |
| 54 |       0.677904 | /home/yzbx/tmp/logs/motion/motion_unet